# Two Stage Sequential Classifier

## Required Imports

In [395]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, plot_confusion_matrix, precision_score, recall_score, roc_curve
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.utils import resample
from sklearn.utils.multiclass import unique_labels

## User Defined Functions

In [ ]:
def create_confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None, display_labels=None, include_values=True, xticks_rotation='horizontal', values_format=None, cmap='viridis', ax=None, colorbar=True):
    
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    
    if display_labels is None:
        if labels is None:
            display_labels = unique_labels(y_true, y_pred)
        else:
            display_labels = labels

    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=display_labels)
    return disp.plot(include_values=include_values,
                     cmap=cmap, ax=ax, xticks_rotation=xticks_rotation,
                     values_format=values_format)

In [ ]:
def specificity_score(y_true, y_pred):
    TN = 0
    FP = 0
    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y_true[i] == 0:
            TN += 1
        if y_pred[i] == 1 and y_true[i] == 0:
            FP += 1
    
    return TN / (TN + FP)

In [ ]:
def positive_pv_score(y_true, y_pred):
    TP = 0
    FP = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i] == 1:
            TP += 1
        elif y_true[i] == 0 and y_pred[i] == 1:
            FP += 1

    return TP/(TP+FP)

In [ ]:
def negative_pv_score(y_true, y_pred):
    TN = 0
    FN = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i] == 0:
            TN += 1
        elif y_true[i] == 1 and y_pred[i] == 0:
            FN += 1
    
    return TN/(TN+FN) 

In [ ]:
def create_ci(bootstrapped_scores, name):
    for i in range(3):
        sorted_scores = np.array(bootstrapped_scores)[:, i]
        sorted_scores.sort()
    
        confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
    
        print("95% Confidence interval for the {} score for class {}: [{:0.4f} - {:0.4}]".format(name, i,
        confidence_lower, confidence_upper))

In [ ]:
def my_classification_report(y_test_np, y_pred):

    tot_TN = 0
    tot_TP = 0
    tot_FP = 0
    tot_FN = 0

    spec_arr = []
    ppv_arr = []
    npv_arr = []
    rec_arr = []
    acc_arr = []

    print("Specificities")
    for group in range(3):
        TN = 0
        TP = 0
        FN = 0
        FP = 0
        for i in range(len(y_test_np)):
            if y_pred[i] != group and y_test_np[i] != group:
                TN += 1
            if y_pred[i] == group and y_test_np[i] != group:
                FP += 1
        tot_TN += TN
        tot_FP += FP
        spec_arr.append(TN/(TN+FP))
        print(group, round(spec_arr[group],4))

    print("Macro Avg : " + str.format('{0:.4f}', np.array(spec_arr).mean()))
    print("Micro Avg : " + str.format('{0:.4f}', tot_TN / (tot_TN + tot_FP)), '\n')

    tot_TN = 0
    tot_TP = 0
    tot_FP = 0
    tot_FN = 0

    print("PPV/Precision")
    for group in range(3):
        TN = 0
        TP = 0
        FN = 0
        FP = 0
        for i in range(len(y_test_np)):
            if y_pred[i] == group and y_test_np[i] == group:
                TP += 1
            if y_pred[i] == group and y_test_np[i] != group:
                FP += 1
        tot_TP += TP
        tot_FP += FP
        ppv_arr.append(TP/(TP+FP))
        print(group, round(ppv_arr[group],4))
    
    print("Macro Avg : " + str.format('{0:.4f}', np.array(ppv_arr).mean()))
    prec_for_mic_f1 = tot_TP / (tot_TP + tot_FP)
    print("Micro Avg : " + str.format('{0:.4f}', prec_for_mic_f1), '\n')

    tot_TN = 0
    tot_TP = 0
    tot_FP = 0
    tot_FN = 0

    print("NPV")
    for group in range(3):
        TN = 0
        TP = 0
        FN = 0
        FP = 0
        for i in range(len(y_test_np)):
            if y_pred[i] != group and y_test_np[i] != group:
                TN += 1
            if y_pred[i] != group and y_test_np[i] == group:
                FN += 1
        tot_TN += TN
        tot_FN += FN
        npv_arr.append(TN/(TN+FN))
        print(group, round(npv_arr[group],4))
    
    print("Macro Avg : " + str.format('{0:.4f}', np.array(npv_arr).mean()))
    print("Micro Avg : " + str.format('{0:.4f}', tot_TN / (tot_TN + tot_FN)), '\n') 
    
    tot_TN = 0
    tot_TP = 0
    tot_FP = 0
    tot_FN = 0

    print("Recall")
    for group in range(3):
        TN = 0
        TP = 0
        FN = 0
        FP = 0
        for i in range(len(y_test_np)):
            if y_pred[i] == group and y_test_np[i] == group:
                TP += 1
            if y_pred[i] != group and y_test_np[i] == group:
                FN += 1
        tot_TP += TP
        tot_FN += FN
        rec_arr.append(TP/(TP+FN))
        print(group, round(rec_arr[group],4))
    
    print("Macro Avg : " + str.format('{0:.4f}', np.array(rec_arr).mean()))
    rec_for_mic_f1 = tot_TP / (tot_TP + tot_FN)
    print("Micro Avg : " + str.format('{0:.4f}', rec_for_mic_f1), '\n')
    
    print("F1 Score")
    for group in range(3):
        if rec_arr[group] + ppv_arr[group] == 0:
            f1 = 0
        else:
            f1 = 2 * (rec_arr[group] * ppv_arr[group]) / (rec_arr[group] + ppv_arr[group])
        print(group, round(f1, 4))
    print("Macro Avg : " + str.format('{0:.4f}', 2 * (np.array(rec_arr).mean() * np.array(ppv_arr).mean()) 
                               / (np.array(rec_arr).mean() + np.array(ppv_arr).mean())))
    print("Micro Avg : " + str.format('{0:.4f}', 2 * (prec_for_mic_f1 * rec_for_mic_f1)/ (prec_for_mic_f1 + rec_for_mic_f1)), '\n')
    
    tot_TN = 0
    tot_TP = 0
    tot_FP = 0
    tot_FN = 0   
    
    print("Accuracy:")
    for group in range(3):
        TN = 0
        TP = 0
        FN = 0
        FP = 0
        for i in range(len(y_test_np)):
            if y_pred[i] == group and y_test_np[i] == group:
                TP += 1
            if y_pred[i] != group and y_test_np[i] != group:
                TN += 1
            if y_pred[i] == group and y_test_np[i] != group:
                FP += 1
            if y_pred[i] != group and y_test_np[i] == group:
                FN += 1
        
        tot_TP += TP
        tot_TN += TN
        tot_FP += FP
        tot_FN += FN
        
        acc_arr.append((TP + TN)/(TP + TN + FP + FN))
        print(group, round(acc_arr[group],10))

    print("Macro Avg : " + str.format('{0:.4f}', np.array(acc_arr).mean()))
    print("Micro Avg : " + str.format('{0:.4f}', (tot_TP + tot_TN)/(tot_TP + tot_TN + tot_FP + tot_FN)), '\n') ##

    print('multiclass acc:', (y_test_np == y_pred).sum() / len(y_test_np))

In [ ]:
def print_metrics(y_test, y_pred):
    print("Sens Score: " + str.format('{0:.4f}', (recall_score(y_test, y_pred))))
    print("Spec Score: " + str.format('{0:.4f}', (specificity_score(y_test.to_numpy(), y_pred))))
    print("PPV Score: " + str.format('{0:.4f}', (positive_pv_score(y_test.to_numpy(), y_pred))))
    print("NPV Score: " + str.format('{0:.4f}', (negative_pv_score(y_test.to_numpy(), y_pred))))
    print("Acc Score: " + str.format('{0:.4f}', (accuracy_score(y_test, y_pred))))
    print("F1 Score: " + str.format('{0:.4f}', (f1_score(y_test, y_pred))))

In [ ]:
def plot_roc_curve(model_name):
    sns.set()
    plt.figure(figsize=(7, 7))

    ns_preds = [0 for _ in range(len(y_test))]

    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_preds)

    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(fpr, tpr, marker='.', label=model_name)
    plt.xlabel('1 - Specificity (False Positive Rate)',fontsize=16)
    plt.ylabel('Sensitivity (True Positive Rate)',fontsize=16)

    plt.legend(loc='lower right')
    plt.title(f'{model_name}: ROC Curve for Test Set', fontsize=20, fontweight="semibold")
    short_auc = round(auc,4)
    plt.text(.93,.1, "AUC: " + str(short_auc), 
        horizontalalignment="center", verticalalignment="center",
        fontsize=14, fontweight="semibold")
    
    plt.show()

In [ ]:
def create_intermed_confusion_matrix(model_name, model, X, y, normalize=None):
    matrix = plot_confusion_matrix(model, X, y,
                               cmap=plt.cm.Blues,
                                  normalize=normalize)
    plt.title(f'{model_name} Confusion Matrix')
    plt.show(matrix)
    plt.show()

In [ ]:
def grid_search(model, params, X_train, y_train):
    grid = GridSearchCV(estimator=model,
                       param_grid=params,
                       scoring='accuracy',
                       cv=5,
                       n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_params_

## Data Loading

In [ ]:
# load data
df = pd.read_csv("features-n.csv")
df.head()

In [ ]:
# obtain list of features selected from LASSO
reduced_features_list = ['t1_log-sigma-1-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis', 't1_log-sigma-3-mm-3D_firstorder_Median', 't1_log-sigma-3-mm-3D_glcm_ClusterProminence', 't1_log-sigma-3-mm-3D_glrlm_LowGrayLevelRunEmphasis', 't1_log-sigma-5-mm-3D_glszm_GrayLevelVariance', 't1_original_glszm_LargeAreaHighGrayLevelEmphasis', 't1_original_glszm_LargeAreaLowGrayLevelEmphasis', 't1_original_shape_Flatness', 't1_wavelet-HHL_firstorder_Skewness', 't1_wavelet-HLL_firstorder_Mean', 't1_wavelet-HLL_glcm_Correlation', 't1_wavelet-LHL_glcm_Correlation', 't1_wavelet-LLH_glszm_LargeAreaLowGrayLevelEmphasis', 't1_wavelet-LLL_firstorder_Skewness', 't1_wavelet-LLL_glszm_ZoneVariance', 't2_log-sigma-1-mm-3D_firstorder_Mean', 't2_log-sigma-1-mm-3D_firstorder_Median', 't2_log-sigma-1-mm-3D_glcm_Correlation', 't2_log-sigma-5-mm-3D_firstorder_Kurtosis', 't2_log-sigma-5-mm-3D_glrlm_RunLengthNonUniformity', 't2_log-sigma-5-mm-3D_glszm_SmallAreaEmphasis', 't2_original_glszm_GrayLevelNonUniformity', 't2_wavelet-HHH_glrlm_RunPercentage', 't2_wavelet-HHH_glszm_LargeAreaEmphasis', 't2_wavelet-HHH_glszm_SmallAreaLowGrayLevelEmphasis', 't2_wavelet-HHL_glcm_ClusterProminence', 't2_wavelet-HHL_glrlm_LongRunHighGrayLevelEmphasis', 't2_wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis', 't2_wavelet-HLH_glszm_SizeZoneNonUniformityNormalized', 't2_wavelet-HLL_firstorder_Mean', 't2_wavelet-LHH_firstorder_Mean', 't2_wavelet-LHH_glszm_GrayLevelNonUniformityNormalized', 't2_wavelet-LHL_glcm_Idn', 't2_wavelet-LLL_firstorder_Energy', 't2_wavelet-LLL_glcm_JointAverage', 't1_log-sigma-3-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis', 't1_wavelet-HHH_firstorder_Skewness', 't1_wavelet-HHH_glszm_LargeAreaHighGrayLevelEmphasis', 't1_wavelet-HHL_glcm_ClusterProminence', 't1_wavelet-LHH_firstorder_Median', 't2_log-sigma-5-mm-3D_glcm_Imc2', 't2_original_glcm_Idmn', 't2_wavelet-LLL_firstorder_Median', 't2_wavelet-HHH_firstorder_Skewness', 't2_original_firstorder_Kurtosis', 't2_wavelet-LLL_firstorder_TotalEnergy']

## NN1: group3+4 vs rest (First Sequential)

### Data Preprocessing

In [ ]:
first_features = df[reduced_features_list]

In [ ]:
# Binarizing target
first_target = df["molecular"]

# Marking 0 as group3+4 and 1 as wnt
first_target = first_target.map(dict(group3=0, group4=0, shh=1, wnt=1))
print(first_target.value_counts())

In [ ]:
# Train/test split
train_idx = df['split'] != 'test'
test_idx = ~train_idx
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(first_features, first_target, test_size = 0.25, random_state = 42)
X_train_1 = first_features[train_idx]
y_train_1 = first_target[train_idx]
X_test_1 = first_features[test_idx]
y_test_1 = first_target[test_idx]
print("TRAIN")
print(y_train_1.value_counts())

print("TEST")
print(y_test_1.value_counts())

In [ ]:
# #NIR
# NIR_G34 = 42/66
# NIR_SHH = 14/66
# NIR_WNT = 10/66
#
# print('%.4f' % NIR_G34)
# print('%.4f' % NIR_SHH)
# print('%.4f' % NIR_WNT)

#### Resampling to Correct for Imbalance

In [ ]:
first_X = pd.concat([X_train_1, y_train_1], axis = 1)
first_X_0 = first_X[first_X['molecular'] == 0]
first_X_1 = first_X[first_X['molecular'] == 1]

In [ ]:
assert len(first_X_0) >= len(first_X_1)
upsampled_1 = resample(first_X_1, replace = True, n_samples = len(first_X_0), random_state = 42)
upsampled = pd.concat([upsampled_1, first_X_0])
upsampled = upsampled.sample(frac = 1, random_state = 42)

In [ ]:
X_train_1 = upsampled.iloc[:, :-1]
y_train_1 = upsampled.iloc[:, -1]
print(y_train_1.value_counts())

#### Standardizing Features

In [ ]:
first_names = X_train_1.columns
first_scaler = preprocessing.StandardScaler()

In [ ]:
X_train_1 = first_scaler.fit_transform(X_train_1)
X_train_1 = pd.DataFrame(X_train_1, columns = first_names)

In [ ]:
X_test_1 = first_scaler.transform(X_test_1)
X_test_1 = pd.DataFrame(X_test_1, columns = first_names)

#### Grid Search for Hyperparameters

In [ ]:
nn_model_1_model = MLPClassifier(max_iter = 2000, random_state = 42)
nn_grid_params_1 = {'hidden_layer_sizes': [(100, 100, 50), (50, 100, 50), (100, 50, 100)],
              'learning_rate': ['constant', 'invscaling', 'adaptive']
}

nn_params_1 = grid_search(nn_model_1_model, nn_grid_params_1, X_train_1, y_train_1)
print(nn_params_1)

#### Model Performance

In [ ]:
nn_seq_1_model = MLPClassifier(nn_params_1['hidden_layer_sizes'], max_iter=2000, random_state=42, learning_rate=nn_params_1['learning_rate'])
nn_seq_1_model.fit(X_train_1, y_train_1)
y_pred_1 = nn_seq_1_model.predict(X_test_1)
print(X_test_1.shape)
print(y_pred_1)

In [ ]:
print_metrics(y_test_1, y_pred_1)

# NN2: SHH vs WNT (Second Sequential) [decomposed from rest1]

### Data Preprocessing

In [ ]:
# obtain list of features selected from LASSO
second_reduced_features_list = ['t1_log-sigma-3-mm-3D_glszm_GrayLevelNonUniformityNormalized', 't1_wavelet-LHL_glcm_Correlation', 't1_wavelet-LHL_glcm_Idn', 't2_wavelet-HHL_firstorder_InterquartileRange', 't2_wavelet-LLL_firstorder_Kurtosis', 't2_log-sigma-1-mm-3D_glszm_HighGrayLevelZoneEmphasis']
print(len(second_reduced_features_list))

In [ ]:
# second_features = df.drop(['molecular','molec_id', 'dx_date','alive','os','pfs','seg_id'], axis = 1)
# second_features = second_features[second_reduced_features_list]
second_df = df[df['molecular'].isin(['wnt', 'shh'])]
second_features = second_df[second_reduced_features_list]

In [ ]:
# Binarizing target
second_target = second_df["molecular"]

# Marking 0 as shh and 1 as wnt
second_target = second_target.map({
    'shh': 0,
    'wnt': 1,
})
print(second_target.value_counts())

In [ ]:
# second_target = second_target.reset_index(drop = True)
# second_features = second_features.reset_index(drop = True)

In [ ]:
# Train/test split
X_train_2 = second_features[train_idx]
X_test_2 = second_features[test_idx]
y_train_2 = second_target[train_idx]
y_test_2 = second_target[test_idx]
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(second_features, second_target,
#                                                     test_size = 0.25, random_state = 42)
print("TRAIN")
print(y_train_2.value_counts())

print("TEST")
print(y_test_2.value_counts())

#### Resampling to Correct for Imbalance 

In [ ]:
second_X = pd.concat([X_train_2, y_train_2], axis = 1)
second_X_0 = second_X[second_X['molecular'] == 0]
second_X_1 = second_X[second_X['molecular'] == 1]

In [ ]:
assert len(second_X_0) >= len(second_X_1)
second_upsampled_1 = resample(second_X_1, replace = True, n_samples = len(second_X_0), random_state = 42)
second_upsampled = pd.concat([second_upsampled_1, second_X_0])
second_upsampled = second_upsampled.sample(frac = 1, random_state = 42)

In [ ]:
X_train_2 = second_upsampled.iloc[:, :-1]
y_train_2 = second_upsampled.iloc[:, -1]
print(y_train_2.value_counts())

#### Standardizing Features

In [ ]:
second_names = X_train_2.columns
second_scaler = preprocessing.StandardScaler()

In [ ]:
X_train_2 = second_scaler.fit_transform(X_train_2)
X_train_2 = pd.DataFrame(X_train_2, columns = second_names)

In [ ]:
X_test_2 = second_scaler.transform(X_test_2)
X_test_2 = pd.DataFrame(X_test_2, columns = second_names)

### Modeling

#### Model Performance

In [ ]:
nn_model_2_model = MLPClassifier(max_iter=2000, random_state=42)
nn_grid_params_2 = {'hidden_layer_sizes': [(100, 100, 50), (50, 100, 50), (100, 50, 100)],
              'learning_rate': ['constant', 'invscaling', 'adaptive']
}

nn_params_2 = grid_search(nn_model_2_model, nn_grid_params_2, X_train_2, y_train_2)
print(nn_params_2)

In [ ]:
nn_seq_2_model = MLPClassifier(nn_params_2['hidden_layer_sizes'], max_iter=2000, random_state=42, learning_rate=nn_params_2['learning_rate'])
nn_seq_2_model.fit(X_train_2, y_train_2)
y_pred_2 = nn_seq_2_model.predict(X_test_2)

In [ ]:
create_intermed_confusion_matrix('Neural Network', nn_seq_2_model, X_test_2, y_test_2)

## Testing

In [ ]:
# Test features
# test_features = df.drop(['molecular','molec_id', 'dx_date','alive','os','pfs','seg_id'], axis = 1)
test_features = df.copy()
test_target = df['molecular']
# test_target = test_target.map(dict(group3 = 0, shh = 1, wnt = 2))
test_target = test_target.map({
    'group3': 0,
    'group4': 0,
    'shh': 1,
    'wnt': 2,
})

In [ ]:
# Train/test split
# X_train, X_test, y_train, y_test = train_test_split(test_features, test_target,
#                                                     test_size = 0.25, random_state = 42)
X_train = test_features[train_idx]
X_test = test_features[test_idx]
y_train = test_target[train_idx]
y_test = test_target[test_idx]
print("TRAIN")
print(y_train.value_counts())

print("TEST")
print(y_test.value_counts())

In [ ]:
# Data Preparation
# X_test = X_test.reset_index(drop = True)
# y_test = y_test.reset_index(drop = True)
# sex_binarized = X_test['sex'].map(dict(M = 1, F = 0)).to_numpy()
# X_test['sex'] = sex_binarized
X_test_reduced = X_test[reduced_features_list]

### Standardization for NN1

In [ ]:
names = X_test_reduced.columns
X_test_for_first = first_scaler.transform(X_test_reduced)
X_test_for_first = pd.DataFrame(X_test_for_first, columns = names)

### Modeling

In [ ]:
y_preds_after_first = nn_seq_1_model.predict(X_test_for_first)
print(y_preds_after_first)

In [ ]:
#keeps all that are group3+4, indifferent to shh/wnt label
group3_indices = np.where(y_preds_after_first == 0)
other_indices = np.where(y_preds_after_first != 0)

group3_preds = y_preds_after_first[group3_indices]
y_test_for_group3 = np.array(y_test)[group3_indices]

X_test_after_first_model = X_test.iloc[other_indices].reset_index(drop = True)
y_test_after_first_model = y_test.iloc[other_indices].reset_index(drop = True)

X_test_after_first_model = X_test_after_first_model[second_reduced_features_list]

### Standardization for NN2

In [ ]:
X_test_after_first_model = second_scaler.transform(X_test_after_first_model)

In [ ]:
#relabel original test-dictionary
#allows us to dro group3 from 2nd stage
#allows accuracy_score of binary outputs: shh now 0, wnt now 1
y_test_after_first_model = np.array(y_test_after_first_model)
y_test_after_first_model[y_test_after_first_model == 0] = -1
y_test_after_first_model[y_test_after_first_model == 1] = 0
y_test_after_first_model[y_test_after_first_model == 2] = 1

In [ ]:
y_preds_after_second = nn_seq_2_model.predict(X_test_after_first_model)
print(y_preds_after_second)
print(accuracy_score(y_test_after_first_model, y_preds_after_second))

### Post-processing

In [ ]:
#relabel prediction outputs to match 3-way; wnt to 2, shh to 1
#group3 is 0 already from "group3_preds" and "y_test_for_group3" derived from "group3_indices"
y_preds_after_second[y_preds_after_second == 1] = 2
y_preds_after_second[y_preds_after_second == 0] = 1

In [ ]:
#revert test-labels to 3-way; wnt back to 2, shh back to 1, group3 back to 0
y_test_after_first_model[y_test_after_first_model == 1] = 2
y_test_after_first_model[y_test_after_first_model == 0] = 1
y_test_after_first_model[y_test_after_first_model == -1] = 0

In [ ]:
y_preds_overall = np.concatenate([y_preds_after_second, group3_preds])
y_test_overall = np.concatenate([y_test_after_first_model, y_test_for_group3])

## Results 

In [ ]:
print("Acc: " + str(accuracy_score(y_test_overall, y_preds_overall)))

In [ ]:
matrix = create_confusion_matrix(y_test_overall, y_preds_overall, cmap=plt.cm.Blues)
plt.title('NN1 + NN2 Confusion Matrix')
plt.show(matrix)
plt.show()

In [ ]:
matrix = create_confusion_matrix(y_test_overall, y_preds_overall, cmap=plt.cm.Blues, normalize = 'true')
plt.title('Normalized NN1 + NN2 Confusion Matrix')
plt.show(matrix)
plt.show()

In [ ]:
# 3-way scores
y_test_np = y_test.to_numpy()
my_classification_report(y_test_overall, y_preds_overall)

y_test_bin = label_binarize(y_test_overall, classes = [0, 1, 2])
y_pred_bin = label_binarize(y_preds_overall, classes = [0, 1, 2])

### Confidence Intervals

In [ ]:
n_classes = 3

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_auc_scores = []
bootstrapped_acc_scores = []
bootstrapped_spec_scores = []
bootstrapped_prec_scores = []
bootstrapped_rec_scores = []
bootstrapped_f_one_scores = []
bootstrapped_ppv_scores = [] ##
bootstrapped_npv_scores = [] ##

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_test_overall), len(y_test_overall))
    if len(np.unique(y_test_overall[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue
    
    one_acc = []
    one_spec = []
    one_prec = []
    one_rec = []
    one_f_one = []
    one_ppv = [] ##
    one_npv = [] ##
    for i in range(n_classes):
        one_acc.append(accuracy_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices]))
        one_spec.append(specificity_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices]))
        one_prec.append(precision_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices]))
        one_rec.append(recall_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices]))
        one_f_one.append(f1_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices]))
        one_ppv.append(positive_pv_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices])) ##
        one_npv.append(negative_pv_score(y_test_bin[:, i][indices], y_pred_bin[:, i][indices])) ##

    bootstrapped_acc_scores.append(one_acc)
    bootstrapped_spec_scores.append(one_spec)
    bootstrapped_prec_scores.append(one_prec)
    bootstrapped_rec_scores.append(one_rec)
    bootstrapped_f_one_scores.append(one_f_one)
    bootstrapped_ppv_scores.append(one_ppv) ##
    bootstrapped_npv_scores.append(one_npv) ##

In [ ]:
create_ci(bootstrapped_acc_scores, "Accuracy")
create_ci(bootstrapped_spec_scores, "Specificity")
create_ci(bootstrapped_prec_scores, "Precision")
create_ci(bootstrapped_rec_scores, "Recall")
create_ci(bootstrapped_f_one_scores, "F1")
create_ci(bootstrapped_ppv_scores, "PPV") ##
create_ci(bootstrapped_npv_scores, "NPV") ##

## Individual Classification Results

In [ ]:
metadata_df = df['molecular']
target_for_table = df['molecular']

In [ ]:
# features_for_table = df.drop(['molecular','molec_id', 'dx_date','alive','os','pfs','seg_id'], axis = 1)
features_for_table = df.drop(columns='molecular')
# target_for_table = target_for_table.map(dict(group3=0, shh = 1, wnt = 2))
target_for_table = target_for_table.map({
    'group3': 0,
    'group4': 0,
    'shh': 1,
    'wnt': 2,
})

In [ ]:
#only used for metadata for which is train and which is test
# X_train_for_table, X_test_for_table, _, _ = train_test_split(features_for_table, target_for_table,
#                                                     test_size = 0.25, random_state = 42)
X_train_for_table = features_for_table[train_idx]
X_test_for_table = features_for_table[test_idx]

In [ ]:
X_train_for_table['Set'] = 'training'
X_test_for_table['Set'] = 'test'

In [ ]:
X_for_table = pd.concat([X_train_for_table, X_test_for_table])
X_for_table = pd.merge(metadata_df, X_for_table, left_index = True, right_index = True)

In [ ]:
final_df = X_for_table[['molecular', 'Set']]
# features_for_table = df.drop(['molecular','molec_id', 'dx_date','alive','os','pfs','seg_id'], axis = 1)
features_for_table = df.drop(columns='molecular')
target_for_table = X_for_table['molecular']
target_for_table = target_for_table.map(dict(group3 = 0, shh = 1, wnt = 2))

In [ ]:
X_table_reduced = features_for_table[reduced_features_list]

In [ ]:
# sex_binarized = X_table_reduced['sex'].map(dict(M = 1, F = 0)).to_numpy()
#
# X_table_reduced['sex'] = sex_binarized

In [ ]:
names = X_table_reduced.columns
X_table_for_first = first_scaler.transform(X_table_reduced)
X_table_for_first = pd.DataFrame(X_table_for_first, columns = names)

In [ ]:
y_probs_after_first_table = nn_seq_1_model.predict_proba(X_table_for_first)

In [ ]:
rf_1_table = pd.DataFrame(y_probs_after_first_table, columns = ['NN1 Group3 Prob','NN1 Non-Group3 Prob'])

In [ ]:
final_df['NN1 Group3 Prob'] = rf_1_table['NN1 Group3 Prob']
final_df['NN1 Non-Group3 Prob'] = rf_1_table['NN1 Non-Group3 Prob']

In [ ]:
y_preds_after_first_table = nn_seq_1_model.predict(X_table_for_first)
wnt_table_indices = np.where(y_preds_after_first_table == 0)
other_table_indices = np.where(y_preds_after_first_table != 0)

In [ ]:
y_table_for_wnt = np.array(y_preds_after_first_table)[wnt_table_indices]
X_table_after_first_model = features_for_table.iloc[other_table_indices]
X_table_after_first_model = X_table_after_first_model[second_reduced_features_list]
X_table_after_first_model_arr = second_scaler.transform(X_table_after_first_model)

In [ ]:
y_probs_after_second_table = nn_seq_2_model.predict_proba(X_table_after_first_model_arr)
y_preds_after_second_table = nn_seq_2_model.predict(X_table_after_first_model_arr)

In [ ]:
nn_2_table = pd.DataFrame(y_probs_after_second_table, columns = ['NN2 SHH Prob','NN2 WNT Prob'])

In [ ]:
X_table_after_first_model['NN2 SHH Prob'] = np.array(nn_2_table['NN2 SHH Prob'])
X_table_after_first_model['NN2 WNT Prob'] = np.array(nn_2_table['NN2 WNT Prob'])

In [ ]:
X_table_after_first_model = X_table_after_first_model[['NN2 SHH Prob', 'NN2 WNT Prob']]

In [ ]:
final_df = final_df.merge(X_table_after_first_model, how='left', left_index=True, right_index=True)

In [ ]:
def f(row):
    if row['NN1 Non-Group3 Prob'] < row['NN1 Group3 Prob']:
        val = 'group3'
    else:
        if row['NN2 SHH Prob'] > row['NN2 WNT Prob']:
            val = 'shh'
        else:
            val = 'wnt'
    return val

In [ ]:
# final_df['molec_id'] = df['molec_id']

In [ ]:
final_df['Pred Path'] = final_df.apply(f, axis = 1)

In [ ]:
# Formatting
# final_df.columns = ['True Molec', 'Set', 'NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob', 'Molec Id', 'Pred Molec']
final_df.columns = ['True Molec', 'Set', 'NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob', 'Pred Molec']
cols = final_df.columns.tolist()
cols = ['NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob',
        'Pred Molec', 'True Molec', 'Set']
final_df = final_df[cols]
final_df = final_df.round(4)

In [390]:
# Formatting
# final_df.columns = ['True Molec', 'Set', 'NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob', 'Molec Id', 'Pred Molec']
final_df.columns = ['True Molec', 'Set', 'NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob', 'Pred Molec']
cols = final_df.columns.tolist()
cols = ['NN1 Non-Group3 Prob', 'NN1 Group3 Prob', 'NN2 SHH Prob', 'NN2 WNT Prob',
        'Pred Molec', 'True Molec', 'Set']
final_df = final_df[cols]
final_df = final_df.round(4)

In [392]:
pd.set_option('display.max_rows', None)
print(final_df)

     NN1 Non-Group3 Prob  NN1 Group3 Prob  NN2 SHH Prob  NN2 WNT Prob  \
0                 1.0000           0.0000           NaN           NaN   
1                 0.9905           0.0095           NaN           NaN   
2                 0.9984           0.0016           NaN           NaN   
3                 0.9989           0.0011           NaN           NaN   
4                 0.7932           0.2068           NaN           NaN   
5                 1.0000           0.0000           NaN           NaN   
6                 0.9817           0.0183           NaN           NaN   
7                 0.9971           0.0029           NaN           NaN   
8                 0.0024           0.9976        0.7802        0.2198   
9                 0.9939           0.0061           NaN           NaN   
10                0.9997           0.0003           NaN           NaN   
11                0.9970           0.0030           NaN           NaN   
12                0.9990           0.0010          

In [ ]:
final_df.to_csv('seq_classification_results.csv')